In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
import string
from sklearn import linear_model
import pandas as pd

In [111]:
from csv import reader
import json
import statistics
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        yield l.strip().split(',')

## Read Prediction

In [104]:
bookCount = defaultdict(int)
totalRead = 0
all_items = []

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1
    all_items.append((user,book,_))
    
threshold = totalRead/2*1.5
    
mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > threshold: break


In [105]:
booksPerUser = defaultdict(list)
usersPerBook = defaultdict(list)
for u,b,r in all_items:
    booksPerUser[u].append(b)
    usersPerBook[b].append(u)

In [106]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

def predictReading(u,b):
    threshold = 0.0001
    bookreaduser2 = set(usersPerBook[b])
    book_node = set(booksPerUser[u]) 
    similarity = []

    for i in book_node:
        similarity.append(Jaccard(bookreaduser2,set(usersPerBook[i])))

    if len(similarity) > 0:
        if sum(similarity)/len(similarity)> threshold:
            return 1
        return 0
    else:
        return 0

In [107]:
predictions = open("predictions_Read.csv", 'w')
for l in open("pairs_Read.csv"):
    if l.startswith("userID"):
    #header
        predictions.write(l)
        continue
    u,b = l.strip().split(',')
    
    predictions.write("{},{},{} \n".format(u,b,int(b in return1 and predictReading(u,b) == 1 )))

predictions.close()

## Category Prediction

In [49]:
dataset = []
for i in readGz("train_Category.json.gz"):
    dataset.append(i)


In [50]:
len(dataset)

100000

In [51]:
dataset[0]

{'user_id': 'u75242413',
 'review_id': 'r45843137',
 'rating': 4,
 'review_text': "a clever book with a deeply troubling premise and an intriguing protagonist. Thompson's clean, sparse prose style kept each page feeling light even as some rather heavy existential questions dropped upon them. I enjoyed it. \n and that cover design is boom-pow gorgeous.",
 'n_votes': 1,
 'genre': 'mystery_thriller_crime',
 'genreID': 3}

In [113]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in dataset:
    r = ''.join([c for c in d['review_text'].lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()


In [126]:
words = [x[1] for x in counts[:8000]]
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)
def feature2(datum):
    feat = [0]* (len(words))
    r = ''.join([c for c in datum['review_text'].lower() if not c in punctuation])
    for w in r.split():
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) # offset
    #feat.append(datum["rating"])
    return feat

In [127]:
X = [feature2(d) for d in dataset]
y = [d['genreID'] for d in dataset]
clf = linear_model.LogisticRegression(C = 1, class_weight = "balanced",fit_intercept = False)
clf.fit(X, y)
theta = clf.coef_
predict2 = clf.predict(X)

In [128]:
predict2[:100]

array([3, 1, 2, 2, 2, 4, 2, 0, 0, 2, 2, 0, 3, 2, 0, 4, 2, 2, 2, 2, 4, 3,
       2, 1, 2, 0, 2, 4, 3, 4, 0, 2, 1, 3, 1, 2, 3, 2, 1, 3, 3, 2, 0, 2,
       2, 4, 3, 1, 4, 2, 0, 4, 3, 3, 3, 4, 2, 3, 3, 4, 4, 2, 2, 2, 2, 3,
       3, 4, 3, 4, 3, 2, 3, 4, 4, 2, 2, 2, 2, 2, 4, 2, 3, 1, 2, 2, 3, 2,
       4, 4, 2, 0, 4, 2, 0, 2, 2, 0, 3, 2])

In [129]:
predictions = open("predictions_Category.csv", 'w')
predictions.write("userID,reviewID,prediction\n")

pred2 = []
X3 = []
result2 = []
for l in readGz("test_Category.json.gz"):
    result2.append([l['user_id'],l['review_id']])
    X3.append(feature2(l))
pred2 = clf.predict(X3)
    
for i in range(len(pred2)):
    predictions.write(result2[i][0] + ',' + result2[i][1] + "," + str(pred2[i]) + "\n")
    
predictions.close()